In [1]:
import pandas as pd
import csv
import numpy as np
import os, os.path
import sys

import collections # to count frequency of elements of an array
# to move files from one directory to another
import shutil

In [2]:
param_dir = "/Users/hn/Documents/01_research_data/NASA/parameters/"
plots_dir = "/Users/hn/Documents/01_research_data/NASA/snapshots/TS/06_snapshot_flat_PNG/"

In [3]:
choices_xl = pd.ExcelFile(param_dir + "all_extended.xlsx")
choices_sheet_names = choices_xl.sheet_names  # see all sheet names

response_set_1_xl = pd.ExcelFile(param_dir + "6000responses.xlsx")
response_sheet_names = response_set_1_xl.sheet_names  # see all sheet names

print (choices_sheet_names[:5])
print (response_sheet_names[:5])

['extended_1', 'extended_2', 'extended_3', 'extended_4', 'extended_5']
['Set 1 E', 'Set 2 E', 'Set 3 E', 'Set 4 E', 'Set 5 E']


In [4]:
evaluation_set_csv = pd.read_csv(param_dir + "evaluation_set.csv")
evaluation_set_csv.drop(labels=["ExctAcr"], axis='columns', inplace=True)

In [5]:
print (len(evaluation_set_csv.ID))
print (len(evaluation_set_csv.ID.unique()))

6340
6340


# Read and assemble the choices; i.e. selected fields for survey

In [6]:
survey_fields = pd.DataFrame()
for a_sheet in choices_sheet_names:
    a_choice_sheet = choices_xl.parse(a_sheet)
    survey_fields = pd.concat([survey_fields, a_choice_sheet])

print (survey_fields.shape)
survey_fields.drop(labels=["ExctAcr"], axis='columns', inplace=True)
print (len(survey_fields.ID))
print (len(survey_fields.ID.unique()))


survey_fields_help = survey_fields.copy()
survey_fields_help = survey_fields_help[evaluation_set_csv.columns].copy()

survey_fields_help.sort_values(by=['ID'], inplace=True)
evaluation_set_csv.sort_values(by=['ID'], inplace=True)

survey_fields_help.reset_index(inplace=True, drop=True)
evaluation_set_csv.reset_index(inplace=True, drop=True)

evaluation_set_csv.equals(survey_fields_help)

(6340, 17)
6340
6340


True

In [7]:
survey_fields.head(2)

,Question_in_set,Question_overall,ID,NDVI_TS_Name,corrected_RGB,TOA_RGB,latitude,longitude,Question Text,Text box Text,CropTyp,Irrigtn,DataSrc,Acres,LstSrvD,county
0,1,1,104119_WSDA_SF_2016,AdamBenton2016_104119_WSDA_SF_2016.png,104119_WSDA_SF_2016_46.83273927_-118.91209151_...,104119_WSDA_SF_2016_46.83273927_-118.91209151_...,46.832739,-118.912092,How would you label this field?,Notes if you want to add any.,alfalfa hay,center pivot,wsda,26,2016/08/17 00:00:00,Adams
1,2,2,51403_WSDA_SF_2016,AdamBenton2016_51403_WSDA_SF_2016.png,51403_WSDA_SF_2016_46.75090118_-118.9797641_co...,51403_WSDA_SF_2016_46.75090118_-118.9797641_TO...,46.750901,-118.979764,How would you label this field?,Notes if you want to add any.,alfalfa hay,center pivot,wsda,2,2016/08/03 00:00:00,Adams


In [8]:
survey_fields_help.head(2)

,ID,CropTyp,Irrigtn,DataSrc,Acres,LstSrvD,county
0,100010_WSDA_SF_2017,alfalfa hay,center pivot,wsda,34,2017/09/12,Grant
1,100018_WSDA_SF_2017,apple,drip,wsda,7,2017/05/03,Grant


In [9]:
evaluation_set_csv.head(2)

,ID,CropTyp,Irrigtn,DataSrc,Acres,LstSrvD,county
0,100010_WSDA_SF_2017,alfalfa hay,center pivot,wsda,34,2017/09/12,Grant
1,100018_WSDA_SF_2017,apple,drip,wsda,7,2017/05/03,Grant


## Count number of questions

In [10]:
question_count = 0

for a_choice_sheet in choices_sheet_names:
    
    # read a damn sheet
    a_choice_sheet = choices_xl.parse(a_choice_sheet)

    # add them to the damn list
    question_count += a_choice_sheet.shape[0]

print('There are [{ques_count}] questions.'.format(ques_count=question_count))

There are [6340] questions.


In [11]:
([x for x in response_sheet_names if 'problem' in x.lower()])

['Set 6 E - Problem',
 'Set 15 Problem',
 'Set 20 Problem',
 'Set 21 - Problem',
 'Set 27 Problem',
 'Set 29 Problem',
 'Set 31 Problem',
 'Set 32 Problem',
 'Set 38 Problem',
 'Set 41 Problem',
 'Set 42 Problem',
 'Set 48 Problem']

# Clean Vote count start here:

#### Problems:
 - In the beginning emails were not collected! (mostly alfalfa and apple?)
 - There are some Forms that I still do not have access to. Eshwar has not transferred them to Google Drive despite repeated emails.
 - There are forms with repeated answers and no email.
 - Min has responded sparsley, mostly no email, and problematic forms. 
 - Different email for Hossein. Since I was receiving emails after completing forms, I used a fake email!! no, thank you!
 - Problematic forms were responded by some, and after fix by others. Drop these forms.
 - Not much Mike's vote in wheat. 

In [12]:
## Define the damn output dataframe
output_df = pd.DataFrame(columns=['Form', 'Question', 'ID',
                                  "Hossein", "Supriya", "Kirti", "Mike", "Min"], 
                                   index=range(question_count))
output_df.head(1)
curr_row = 0

extended_choices = pd.DataFrame()

###### populate the output datafrme

for response_sheet_name in response_sheet_names:
    # If "problem" in the name of sheet, pass
    #
    #   IF Kirti responded to a problematic sheet, and then to fixed sheet,
    #   Then we have duplicates!!!!!
    #
    # if 'problem' in response_sheet_name.lower():
    #     continue

    # pick up the numeric part of the sheet names
    sheet_numeric_part = response_sheet_name.split()[1]
    
    # Form sheet names of choices excel sheets
    choice_sheet_name = "extended_" + sheet_numeric_part
    
    a_choice_sheet = choices_xl.parse(choice_sheet_name)
    a_response_sheet = response_set_1_xl.parse(response_sheet_name)

    # If no email is recoded, pass. We do not want it
    # a_response_sheet['Email Address'].isnull().any() works as well. 
    # It is an indication that the form did not collect emails! One or all!
    if a_response_sheet['Email Address'].isnull().all():
        continue

    
    # Fix Hossein email. (replace emails by names!)
    a_response_sheet['Email Address'] = a_response_sheet['Email Address'].str.lower()

    a_response_sheet.loc[a_response_sheet['Email Address'].str.contains('noorazar', na=False), 
                         'Email Address']="Hossein"

    a_response_sheet.loc[a_response_sheet['Email Address'].str.contains('kirti', na=False), 
                         'Email Address']="Kirti"

    a_response_sheet.loc[a_response_sheet['Email Address'].str.contains('supriya', na=False), 
                         'Email Address']="Supriya"

    a_response_sheet.loc[a_response_sheet['Email Address'].str.contains('brady', na=False), 
                         'Email Address']="Mike"

    a_response_sheet.loc[a_response_sheet['Email Address'].str.contains('ming', na=False), 
                         'Email Address']="Min"

    # Fix repeated Kirti in one sheet
    #
    #   IF Kirti responded to a problematic sheet, and then to fixed sheet,
    #   Then we have duplicates!!!!!
    #
    latest_Kirti = a_response_sheet[a_response_sheet['Email Address']=="Kirti"].Timestamp.max()
    Kirti = a_response_sheet[a_response_sheet['Email Address']=="Kirti"]
    # The fact that we have "<" below, eliminates to count how many times
    # Kirti has responded. i.e. if there is one response, it will not be dropped!
    bad_index = Kirti[Kirti.Timestamp < latest_Kirti].index
    a_response_sheet.drop(bad_index, inplace=True)

    latest_Hossein = a_response_sheet[a_response_sheet['Email Address']=="Hossein"].Timestamp.max()
    Hossein = a_response_sheet[a_response_sheet['Email Address']=="Hossein"]
    # The fact that we have "<" below, eliminates to count how many times
    # Kirti has responded. i.e. if there is one response, it will not be dropped!
    bad_index = Hossein[Hossein.Timestamp < latest_Hossein].index
    a_response_sheet.drop(bad_index, inplace=True)
    
    
    latest_Supriya = a_response_sheet[a_response_sheet['Email Address']=="Supriya"].Timestamp.max()
    Supriya = a_response_sheet[a_response_sheet['Email Address']=="Supriya"]
    # The fact that we have "<" below, eliminates to count how many times
    # Kirti has responded. i.e. if there is one response, it will not be dropped!
    bad_index = Supriya[Supriya.Timestamp < latest_Supriya].index
    a_response_sheet.drop(bad_index, inplace=True)
    
    latest_Mike = a_response_sheet[a_response_sheet['Email Address']=="Mike"].Timestamp.max()
    Mike = a_response_sheet[a_response_sheet['Email Address']=="Mike"]
    # The fact that we have "<" below, eliminates to count how many times
    # Kirti has responded. i.e. if there is one response, it will not be dropped!
    bad_index = Mike[Mike.Timestamp < latest_Mike].index
    a_response_sheet.drop(bad_index, inplace=True)
    
    latest_Min = a_response_sheet[a_response_sheet['Email Address']=="Min"].Timestamp.max()
    Min = a_response_sheet[a_response_sheet['Email Address']=="Min"]
    # The fact that we have "<" below, eliminates to count how many times
    # Kirti has responded. i.e. if there is one response, it will not be dropped!
    bad_index = Min[Min.Timestamp < latest_Min].index
    a_response_sheet.drop(bad_index, inplace=True)


    
    # If less than or equal to 2 people responsed, pass. Keep it if 3 people has responded
    if len(a_response_sheet['Email Address']) != len(a_response_sheet['Email Address'].unique()):
        raise ValueError("Something is wrong in email address column")

    # I commented out the following, since some 
    # fucking sheets had problem. So, some people responded to it, but
    # did not respond to fixed Form. And I want to keep as much as possible!
    # i.e. I will keep the problematic forms and keep whatever questions in them.
    # if len(a_response_sheet['Email Address'])<=2:
    #     continue
    
    #
    # drop unwanted opinions, i.e. keep experts opinions
    #
    # a_response_sheet = a_response_sheet[~(a_response_sheet["Email Address"]=="Min")].copy()
    
    for a_col_name in a_response_sheet.columns:
        if "http" in a_col_name:
            question_number = a_col_name.split()[1].split(":")[0]
            currnt_ID = a_choice_sheet.loc[int(question_number)-1, "ID"]
            if currnt_ID in list(output_df.ID):
                curr_idx = output_df[output_df.ID == currnt_ID].index
                
                if "Hossein" in list(a_response_sheet["Email Address"]):
                    output_df.loc[curr_idx, "Hossein"]=a_response_sheet[a_response_sheet["Email Address"] == \
                                                                     "Hossein"][a_col_name].values[0]

                if "Supriya" in list(a_response_sheet["Email Address"]):
                    output_df.loc[curr_idx, "Supriya"]=a_response_sheet[a_response_sheet["Email Address"] == \
                                                                     "Supriya"][a_col_name].values[0]

                if "Kirti" in list(a_response_sheet["Email Address"]):
                    output_df.loc[curr_idx, "Kirti"]=a_response_sheet[a_response_sheet["Email Address"] == \
                                                                     "Kirti"][a_col_name].values[0]
                if "Mike" in list(a_response_sheet["Email Address"]):
                    output_df.loc[curr_idx, "Mike"]=a_response_sheet[a_response_sheet["Email Address"] == \
                                                                     "Mike"][a_col_name].values[0]
                
                if "Min" in list(a_response_sheet["Email Address"]):
                    output_df.loc[curr_idx, "Min"]=a_response_sheet[a_response_sheet["Email Address"] == \
                                                                     "Min"][a_col_name].values[0]
            else:
                output_df.loc[curr_row, "ID"] = currnt_ID
                output_df.loc[curr_row, "Form"] = int(sheet_numeric_part)
                output_df.loc[curr_row, "Question"] = int(question_number)
                
                if "Hossein" in list(a_response_sheet["Email Address"]):
                    output_df.loc[curr_row, "Hossein"]=a_response_sheet[a_response_sheet["Email Address"] == \
                                                                     "Hossein"][a_col_name].values[0]

                if "Supriya" in list(a_response_sheet["Email Address"]):
                    output_df.loc[curr_row, "Supriya"]=a_response_sheet[a_response_sheet["Email Address"] == \
                                                                     "Supriya"][a_col_name].values[0]

                if "Kirti" in list(a_response_sheet["Email Address"]):
                    output_df.loc[curr_row, "Kirti"]=a_response_sheet[a_response_sheet["Email Address"] == \
                                                                     "Kirti"][a_col_name].values[0]
                if "Mike" in list(a_response_sheet["Email Address"]):
                    output_df.loc[curr_row, "Mike"]=a_response_sheet[a_response_sheet["Email Address"] == \
                                                                     "Mike"][a_col_name].values[0]
                
                if "Min" in list(a_response_sheet["Email Address"]):
                    output_df.loc[curr_row, "Min"]=a_response_sheet[a_response_sheet["Email Address"] == \
                                                                     "Min"][a_col_name].values[0]
                curr_row += 1

output_df.dropna(how='all', axis='rows', inplace=True);
print (output_df.shape)
print (len(output_df.ID))
print (len(output_df.ID.unique()))

(5740, 8)
5740
5740


In [13]:
output_df.head(2)

,Form,Question,ID,Hossein,Supriya,Kirti,Mike,Min
0,1,1,104119_WSDA_SF_2016,Single Crop,Single Crop,Single Crop,Single Crop,Single Crop
1,1,2,51403_WSDA_SF_2016,Single Crop,Single Crop,Single Crop,Single Crop,Unsure


In [14]:
output_df.loc[100:102, :]

,Form,Question,ID,Hossein,Supriya,Kirti,Mike,Min
100,11,41,7676_WSDA_SF_2016,Single Crop,Single Crop,Single Crop,Single Crop,NaN
101,11,42,8400_WSDA_SF_2016,Single Crop,Single Crop,Single Crop,Single Crop,NaN
102,11,43,8496_WSDA_SF_2016,Single Crop,Single Crop,Single Crop,Unsure,NaN


In [15]:
output_df.tail(2)

,Form,Question,ID,Hossein,Supriya,Kirti,Mike,Min
5738,106,39,98716_WSDA_SF_2017,Unsure,Double Crop/Cover crop,Double Crop/Cover crop,NaN,NaN
5739,106,40,99604_WSDA_SF_2017,Unsure,Single Crop,Double Crop/Cover crop,NaN,NaN


In [16]:
output_df[output_df.Form==15].head(5)

,Form,Question,ID,Hossein,Supriya,Kirti,Mike,Min
240,15,1,103689_WSDA_SF_2017,Single Crop,Single Crop,Single Crop,Single Crop,Single Crop
241,15,2,103693_WSDA_SF_2017,Single Crop,Single Crop,Single Crop,Single Crop,Single Crop
242,15,3,103771_WSDA_SF_2017,Single Crop,Single Crop,Single Crop,Single Crop,Single Crop
243,15,4,103827_WSDA_SF_2017,Single Crop,Single Crop,Single Crop,Single Crop,Unsure
244,15,5,103902_WSDA_SF_2017,Single Crop,Single Crop,Single Crop,Single Crop,Single Crop


# Correct the damn responses

1. We have not been consistent in the Forms!
2. The options we gave there was horrible. For example, the options were:
   - Single Crop
   - Double Crop
   - Unsure
   - Other:
   
Convert anything that has ```cover```, ```mustard``` in it to ```double-crop```, single crops are single crops, anything else will be ```Unsure```.

If an answer has any of these pairs in it, it will be labeled as unsure ```[single, double]```, ```[single, cover]```,
```[single, mustard]```.

In [17]:
output_df.Supriya.unique()

array(['Single Crop', 'Double Crop',
       'May be mustard growing here second curve from aug-sept onwards',
       'may be mustard',
       'May be non-mustard cover crop suggested by second curve',
       'Looks like a non-mustard cover crop', 'Double Crop/Cover crop',
       nan, 'Unsure'], dtype=object)

In [18]:
output_df.reset_index(inplace=True, drop=True)
output_backup = output_df.copy()

In [19]:
output_df = output_backup.copy()

In [20]:
output_df.Supriya.unique()

array(['Single Crop', 'Double Crop',
       'May be mustard growing here second curve from aug-sept onwards',
       'may be mustard',
       'May be non-mustard cover crop suggested by second curve',
       'Looks like a non-mustard cover crop', 'Double Crop/Cover crop',
       nan, 'Unsure'], dtype=object)

In [21]:
people = ["Hossein", "Supriya", "Kirti", "Mike", "Min"]

# We need to keep "none" so that we can use it to compute
# majority vote where someone has not responded a Form
output_df[people] = output_df[people].fillna('none')

In [22]:
output_df.Supriya.unique()

array(['Single Crop', 'Double Crop',
       'May be mustard growing here second curve from aug-sept onwards',
       'may be mustard',
       'May be non-mustard cover crop suggested by second curve',
       'Looks like a non-mustard cover crop', 'Double Crop/Cover crop',
       'none', 'Unsure'], dtype=object)

In [23]:
for idx in output_df.index:
    for person in people:
        if output_df.loc[idx, person] in ["Single Crop", "Double Crop", "Unsure", "none"]:
            continue
            
        if "cover" in output_df.loc[idx, person].lower() or "mustard" in output_df.loc[idx, person].lower():
            output_df.loc[idx, person] = "Double Crop"
        
        else:
            output_df.loc[idx, person] = "Unsure"

In [24]:
output_df.Supriya.unique()

array(['Single Crop', 'Double Crop', 'none', 'Unsure'], dtype=object)

# Compute Majority and Final Votes

In [25]:
output_df["Vote"] = "Unsure"

In [26]:
for idx in output_df.index:
    valid_votes = output_df.loc[idx, people].values != 'none'
    valid_votes = output_df.loc[idx, people].values[valid_votes]

    needed_vote_count = np.ceil(len(valid_votes)/2)
    valid_votes_freq = collections.Counter(valid_votes)

    for key, value in valid_votes_freq.items():
        if value>=needed_vote_count:
            output_df.loc[idx, "Vote"] = key

In [27]:
output_df.head(2)

,Form,Question,ID,Hossein,Supriya,Kirti,Mike,Min,Vote
0,1,1,104119_WSDA_SF_2016,Single Crop,Single Crop,Single Crop,Single Crop,Single Crop,Single Crop
1,1,2,51403_WSDA_SF_2016,Single Crop,Single Crop,Single Crop,Single Crop,Unsure,Single Crop


In [28]:
output_df.tail(2)

,Form,Question,ID,Hossein,Supriya,Kirti,Mike,Min,Vote
5738,106,39,98716_WSDA_SF_2017,Unsure,Double Crop,Double Crop,none,none,Double Crop
5739,106,40,99604_WSDA_SF_2017,Unsure,Single Crop,Double Crop,none,none,Unsure


In [29]:
output_df[output_df.Vote=="Double Crop"].tail(5)

,Form,Question,ID,Hossein,Supriya,Kirti,Mike,Min,Vote
5732,106,33,100707_WSDA_SF_2017,Double Crop,Double Crop,Double Crop,none,none,Double Crop
5733,106,34,102519_WSDA_SF_2017,Double Crop,Double Crop,Double Crop,none,none,Double Crop
5734,106,35,103372_WSDA_SF_2017,Double Crop,Double Crop,Double Crop,none,none,Double Crop
5736,106,37,106509_WSDA_SF_2017,Single Crop,Double Crop,Double Crop,none,none,Double Crop
5738,106,39,98716_WSDA_SF_2017,Unsure,Double Crop,Double Crop,none,none,Double Crop


In [30]:
output_df[output_df.Vote=="Single Crop"].tail(5)

,Form,Question,ID,Hossein,Supriya,Kirti,Mike,Min,Vote
5709,106,10,48092_WSDA_SF_2018,Single Crop,Single Crop,Single Crop,none,none,Single Crop
5710,106,11,49183_WSDA_SF_2018,Single Crop,Single Crop,Single Crop,none,none,Single Crop
5717,106,18,100904_WSDA_SF_2018,Single Crop,Double Crop,Single Crop,none,none,Single Crop
5730,106,31,110424_WSDA_SF_2018,Single Crop,Single Crop,Single Crop,none,none,Single Crop
5737,106,38,108383_WSDA_SF_2017,Single Crop,Single Crop,Double Crop,none,none,Single Crop


In [31]:
good_columns = ['ID', 'NDVI_TS_Name',
                'CropTyp', 'Irrigtn', 'DataSrc', 'Acres', 
                'LstSrvD', 'county']
survey_fields = survey_fields[good_columns]

extended_output = pd.merge(output_df, survey_fields, on=['ID'], how='left')
extended_output.head(2)

,Form,Question,ID,Hossein,Supriya,Kirti,Mike,Min,Vote,NDVI_TS_Name,CropTyp,Irrigtn,DataSrc,Acres,LstSrvD,county
0,1,1,104119_WSDA_SF_2016,Single Crop,Single Crop,Single Crop,Single Crop,Single Crop,Single Crop,AdamBenton2016_104119_WSDA_SF_2016.png,alfalfa hay,center pivot,wsda,26,2016/08/17 00:00:00,Adams
1,1,2,51403_WSDA_SF_2016,Single Crop,Single Crop,Single Crop,Single Crop,Unsure,Single Crop,AdamBenton2016_51403_WSDA_SF_2016.png,alfalfa hay,center pivot,wsda,2,2016/08/03 00:00:00,Adams


# Drop the fields that are labeled by Experts

#### Read First set

In [32]:
perry_dir = "/Users/hn/Documents/01_research_data/NASA/Perry_and_Co/"
set1_fields = pd.read_csv(perry_dir + "set_1_experts_stats_extended_sortOpinionCrop.csv")
print (set1_fields.shape)
set1_fields.head(2)

(294, 21)


,Form,Question,opinion_count,ID,Perry,Andrew,Tim,Question_in_set,Question_overall,NDVI_TS_Name,...,TOA_RGB,latitude,longitude,CropTyp,Irrigtn,DataSrc,Acres,ExctAcr,LstSrvD,county
0,3,32,3,99837_WSDA_SF_2017,Double Crop,Single Crop,Mustard Crop,32,132,Grant2017_99837_WSDA_SF_2017.png,...,99837_WSDA_SF_2017_46.87015998_-119.40630986_T...,46.87016,-119.406310,barley,center pivot,wsda,73,73.250981,2017/10/10,Grant
1,2,22,3,98359_WSDA_SF_2017,Double Crop,Unsure,Single Crop,22,72,Grant2017_98359_WSDA_SF_2017.png,...,98359_WSDA_SF_2017_46.93911002_-119.65673394_T...,46.93911,-119.656734,"bean, dry",rill,wsda,24,24.163431,2017/07/10,Grant


#### Read Second Set

In [33]:
param_in_data_dir = "/Users/hn/Documents/01_research_data/NASA/parameters/"
hand_picked_set2 = pd.read_csv(param_in_data_dir + "threeHundred_IDs_Set2_Perry.csv")
hand_picked_set2.dropna(inplace=True)
print (hand_picked_set2.shape)
hand_picked_set2.head(2)

(322, 1)


,ID
0,52701_WSDA_SF_2016
1,53739_WSDA_SF_2016


#### Get unique IDs of first and second set

In [34]:
expert_IDs = list(hand_picked_set2.ID) + list(set1_fields.ID)
expert_IDs = list(set(expert_IDs))
len(expert_IDs)

475

In [35]:
print (len(extended_output.ID.unique()))
print (len(extended_output.ID))

5740
5740


In [36]:
single_index = extended_output[extended_output.Vote=="Single Crop"].index
extended_output.loc[single_index, "Vote"] = "single"

double_index = extended_output[extended_output.Vote=="Double Crop"].index
extended_output.loc[double_index, "Vote"] = "double"


In [39]:
extended_output.tail(10)

,Form,Question,ID,Hossein,Supriya,Kirti,Mike,Min,Vote,NDVI_TS_Name,CropTyp,Irrigtn,DataSrc,Acres,LstSrvD,county
5730,106,31,110424_WSDA_SF_2018,Single Crop,Single Crop,Single Crop,none,none,single,FranklinYakima2018_110424_WSDA_SF_2018.png,yellow mustard,sprinkler,wsda,1,2018/10/09 00:00:00,Franklin
5731,106,32,95998_WSDA_SF_2018,Double Crop,Double Crop,Double Crop,none,none,double,FranklinYakima2018_95998_WSDA_SF_2018.png,yellow mustard,center pivot,wsda,3,2018/09/11 00:00:00,Franklin
5732,106,33,100707_WSDA_SF_2017,Double Crop,Double Crop,Double Crop,none,none,double,Grant2017_100707_WSDA_SF_2017.png,yellow mustard,center pivot/sprinkler,wsda,148,2017/10/16,Grant
5733,106,34,102519_WSDA_SF_2017,Double Crop,Double Crop,Double Crop,none,none,double,Grant2017_102519_WSDA_SF_2017.png,yellow mustard,center pivot,wsda,37,2017/10/19,Grant
5734,106,35,103372_WSDA_SF_2017,Double Crop,Double Crop,Double Crop,none,none,double,Grant2017_103372_WSDA_SF_2017.png,yellow mustard,center pivot,wsda,11,2017/10/17,Grant
5735,106,36,104188_WSDA_SF_2017,Unsure,Double Crop,Unsure,none,none,Unsure,Grant2017_104188_WSDA_SF_2017.png,yellow mustard,center pivot,wsda,135,2017/10/17,Grant
5736,106,37,106509_WSDA_SF_2017,Single Crop,Double Crop,Double Crop,none,none,double,Grant2017_106509_WSDA_SF_2017.png,yellow mustard,center pivot,wsda,135,2017/10/17,Grant
5737,106,38,108383_WSDA_SF_2017,Single Crop,Single Crop,Double Crop,none,none,single,Grant2017_108383_WSDA_SF_2017.png,yellow mustard,center pivot,wsda,4,2017/10/17,Grant
5738,106,39,98716_WSDA_SF_2017,Unsure,Double Crop,Double Crop,none,none,double,Grant2017_98716_WSDA_SF_2017.png,yellow mustard,center pivot,wsda,88,2017/10/17,Grant
5739,106,40,99604_WSDA_SF_2017,Unsure,Single Crop,Double Crop,none,none,Unsure,Grant2017_99604_WSDA_SF_2017.png,yellow mustard,center pivot,wsda,152,2017/10/16,Grant


## Drop the fields that are labeled by Experts

In [40]:
nonExpert_extended_output = extended_output[~extended_output.ID.isin(expert_IDs)]

## Keep only limited crops

In [41]:
wanted_crops = ['alfalfa seed',
                'barley', 'barley hay', 'bean, dry', 'bean, green', 'bluegrass seed', 'buckwheat', 
                'canola', 'carrot', 'corn seed', 'corn, field', 'corn, sweet', 
                'grass hay', 'grass seed',
                'market crops', 'mint',
                'oat hay', 'onion',
                'pea seed', 'pea, dry', 'pea, green', 'potato', 
                'triticale',
                'wheat',
                'yellow mustard']

limitCrops_nonExpert_extended = nonExpert_extended_output[nonExpert_extended_output.CropTyp.isin(wanted_crops)]

In [42]:
limitCrops_nonExpert_extended.shape

(1429, 16)

In [43]:
limitCrops_nonExpert_extended.head(2)

,Form,Question,ID,Hossein,Supriya,Kirti,Mike,Min,Vote,NDVI_TS_Name,CropTyp,Irrigtn,DataSrc,Acres,LstSrvD,county
1202,31,3,53189_WSDA_SF_2016,Single Crop,Single Crop,Single Crop,none,none,single,AdamBenton2016_53189_WSDA_SF_2016.png,barley,center pivot,wsda,132,2016/09/20 00:00:00,Adams
1203,31,4,101534_WSDA_SF_2018,Single Crop,Double Crop,Single Crop,none,none,single,FranklinYakima2018_101534_WSDA_SF_2018.png,barley,center pivot,wsda,3,2018/08/14 00:00:00,Franklin


In [45]:
limitCrops_nonExpert_extended.Vote.unique()

array(['single', 'double', 'Unsure'], dtype=object)

# Write to disc only the definite votes; single or double

In [46]:
good_votes = ['single', 'double']

print (limitCrops_nonExpert_extended.shape)
limitCrops_nonExpert_extended = limitCrops_nonExpert_extended[
                                   limitCrops_nonExpert_extended.Vote.isin(good_votes)]
print (limitCrops_nonExpert_extended.shape)

print (nonExpert_extended_output.shape)
nonExpert_extended_output = nonExpert_extended_output[
                                   nonExpert_extended_output.Vote.isin(good_votes)]

print (nonExpert_extended_output.shape)

(1429, 16)
(1387, 16)
(5286, 16)
(5187, 16)


In [47]:
out_dir = "/Users/hn/Documents/01_research_data/NASA/ML_data/"

out_name = out_dir + "nonExpert_votes.csv"
nonExpert_extended_output.to_csv(out_name, index = False)

out_name = out_dir + "limitCrops_nonExpert_votes.csv"
limitCrops_nonExpert_extended.to_csv(out_name, index = False)

In [48]:
limitCrops_nonExpert_extended.head(2)

,Form,Question,ID,Hossein,Supriya,Kirti,Mike,Min,Vote,NDVI_TS_Name,CropTyp,Irrigtn,DataSrc,Acres,LstSrvD,county
1202,31,3,53189_WSDA_SF_2016,Single Crop,Single Crop,Single Crop,none,none,single,AdamBenton2016_53189_WSDA_SF_2016.png,barley,center pivot,wsda,132,2016/09/20 00:00:00,Adams
1203,31,4,101534_WSDA_SF_2018,Single Crop,Double Crop,Single Crop,none,none,single,FranklinYakima2018_101534_WSDA_SF_2018.png,barley,center pivot,wsda,3,2018/08/14 00:00:00,Franklin
